In [ ]:
%%sql
USE assignment_db

In [ ]:
%%sql
WITH dim_times AS ( SELECT (current_date - interval '1 day') as agg_date,
                       (current_date - interval '2 days') as end_date,
                       (current_date - interval '32 days') as start_date
    ),
dims AS (
    SELECT DISTINCT cu.customer_id, i.industry
    FROM industries i 
    LEFT JOIN companies co ON i.industry = co.industry
    LEFT JOIN customers cu ON co.company_name = cu.company_name
),
aux_24 AS (
    SELECT industry, sum(amount) as value_24h
    FROM orders right join dims USING(customer_id)
    JOIN dim_times
    WHERE year = YEAR(agg_date)
    AND month = MONTH(agg_date)
    AND day = DAY(agg_date)
    GROUP BY 1
),
aux_30d_sum AS (
    SELECT industry, year, month, day, sum(amount) as sum_value
    FROM orders
    RIGHT JOIN dims USING(customer_id)
    JOIN dim_times
    WHERE ( (year = YEAR(start_date) AND 
             month = MONTH(start_date) AND
             day >= DAY(start_date))
            OR
           (year = YEAR(start_date) AND 
             month > MONTH(start_date) AND
             month < DAY(end_date))
           OR
           (year = YEAR(end_date) AND 
             month = MONTH(end_date) AND
             day <= DAY(end_date))
          )
    GROUP BY 1, 2, 3, 4
    ),
aux_30d_avg AS (
    SELECT industry, AVG(sum_value) as avg_30d
    FROM aux_30d_sum
    GROUP BY 1
)
select industry,
aux_24.value_24h,
aux_30d_avg.avg_30d,
aux_24.value_24h - aux_30d_avg.avg_30d as diff,
abs(aux_24.value_24h - aux_30d_avg.avg_30d) as abs_diff,
(aux_24.value_24h - aux_30d_avg.avg_30d) / aux_30d_avg.avg_30d * 100 as pct_diff
FROM aux_24 INNER JOIN aux_30d_avg USING(industry)
ORDER BY abs_diff desc
LIMIT 3




